In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [10]:
base = 'http://officequotes.net/'

In [11]:
r = requests.get(base)

In [12]:
soup = BeautifulSoup(r.content, 'html.parser')

In [13]:
all_links = [link.get('href') for link in soup.find_all('a')]

In [14]:
episode_links = all_links[8:-9]

In [15]:
len(episode_links)

186

# Scraping officequotes.net

In [65]:
lines = []
chars = []
eps = []
for i in episode_links:
    ep_lines = []
    ep_characters = []
    ep_name = []
    url = base + i
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    quotes = soup.findAll("div", {"class" : "quote"})
    for quote in quotes:
        for br in quote.findAll('br'):
            ep_lines.append(br.previous_sibling)
        for b in quote.findAll('b'):
            if not b.findAll('u'):
                ep_characters.append(b.string)
                ep_name.append(i)
    lines.append(ep_lines)
    chars.append(ep_characters)
    eps.append(ep_name)
    
  

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [44]:
len(lines), len(chars), len(eps)

(186, 186, 186)

In [66]:
good_idx = []
for e, i in enumerate(lines):
    if len(lines[e]) == len(chars[e]):
        good_idx.append(e)

In [46]:
len(good_idx)

63

In [67]:
lines_good = []
chars_good = []
eps_good = []
for i in good_idx:
    lines_good.append(lines[i])
    chars_good.append(chars[i])
    eps_good.append(eps[i])
    

In [59]:
len(lines_good), len(chars_good), len(eps_good)

(63, 63, 63)

In [68]:
flat_chars_good = [i for sub in chars_good for i in sub]
flat_lines_good = [i for sub in lines_good for i in sub]
flat_eps_good = [i for sub in eps_good for i in sub]

In [69]:
len(flat_chars_good), len(flat_lines_good), len(flat_eps_good)

(20905, 20905, 20905)

In [70]:
df = pd.DataFrame(list(zip(flat_chars_good, flat_lines_good, flat_eps_good)), columns = ['character', 'line', 'episode'])

In [74]:
df.to_csv('office_quotes_raw.csv', index=False)

In [82]:
df['episode'].unique()

array(['no1-03.php', 'no1-04.php', 'no2-03.php', 'no2-05.php',
       'no2-10.php', 'no2-11.php', 'no2-13.php', 'no2-14.php',
       'no2-15.php', 'no2-16.php', 'no2-17.php', 'no2-18.php',
       'no2-19.php', 'no2-21.php', 'no2-22.php', 'no3-06.php',
       'no3-07.php', 'no3-09.php', 'no3-10.php', 'no3-11.php',
       'no3-12.php', 'no3-14.php', 'no3-15.php', 'no3-17.php',
       'no3-18.php', 'no3-22.php', 'no3-23.php', 'no4-02.php',
       'no4-03.php', 'no4-04.php', 'no4-10.php', 'no4-11.php',
       'no5-02.php', 'no5-04.php', 'no5-05.php', 'no5-10.php',
       'no5-13.php', 'no5-14.php', 'no5-15.php', 'no5-16.php',
       'no5-17.php', 'no5-23.php', 'no5-24.php', 'no5-25.php',
       'no6-03.php', 'no6-04.php', 'no6-06.php', 'no6-07.php',
       'no6-08.php', 'no6-09.php', 'no6-10.php', 'no6-12.php',
       'no6-20.php', 'no6-22.php', 'no7-02.php', 'no7-04.php',
       'no7-05.php', 'no7-06.php', 'no7-07.php', 'no7-08.php',
       'no7-10.php', 'no7-12.php', 'no7-18.php'], dtype

In [89]:
last_line = []
episode = []
char = []
for i in df['episode'].unique():
    last_line.append(df[df['episode']==i]['line'].values[-1])
    char.append(df[df['episode']==i]['character'].values[-1])
    episode.append(df[df['episode']==i]['episode'].values[-1])
    
df_check = pd.DataFrame(list(zip(char, last_line, episode)), columns = ['char', 'last_line', 'episode'])

In [91]:
df_check.to_csv('spotcheck.csv', index=False)

In [2]:
df = pd.read_csv('office_quotes_raw.csv')
df['character'] = df['character'].str.replace('[^a-zA-Z]', '')
df['line'] = df['line'].str.replace('\[.*?\]', '')

In [3]:
df.head()

,character,line,episode
0,Michael,Pam. Pamela. Pam-elama-ding-dong. Making copies.,no1-03.php
1,Pam,I'm not making any copies.,no1-03.php
2,Michael,"Let's go. Messages. Stat. Lots to do, lots to...",no1-03.php
3,Pam,Nothing new.,no1-03.php
4,Michael,Lay them on me. What?,no1-03.php
